In [ ]:
import healer

In [ ]:
resp = healer.generate_filepath_json(prompt="give me a synthetic sample response for the rulefile updater.", model="gpt-oss-120b")
resp

In [1]:
import os
import json
from pathlib import Path

from healer import generate_filepath_json
from index import load_transcript

base_dir = Path('/Users/darshil/projects/claude-heal')
sample_dir = base_dir / 'sample_jsonls'
paths_file = sample_dir / 'claudemd_paths.txt'

# pick the smallest JSONL file in sample_jsonls
jsonl_files = [p for p in sample_dir.glob('*.jsonl') if p.is_file()]
if not jsonl_files:
    raise RuntimeError('No JSONL files found in sample_jsonls')
smallest_jsonl = sorted(jsonl_files, key=lambda p: p.stat().st_size)[4]

# load and clean the JSONL content using the cleanup logic from index.py
cleaned_records = load_transcript(str(smallest_jsonl))
cleaned_jsonl_content = "\n".join(json.dumps(record, ensure_ascii=False) for record in cleaned_records)

# read all CLAUDE.md files listed in the paths file
claude_md_paths = []
with paths_file.open('r', encoding='utf-8') as f:
    for line in f:
        p = line.strip()
        if not p:
            continue
        claude_md_paths.append(Path(p))

claude_blurbs = []
for p in claude_md_paths:
    try:
        # read at most first 400 chars to avoid huge prompts from CLAUDE.md files
        with p.open('r', encoding='utf-8') as f:
            content = f.read()
            claude_blurbs.append(f"PATH: {p}\n---\n{content}\n")
    except Exception as e:
        claude_blurbs.append(f"PATH: {p}\n---\n<error reading: {e}>\n")

prompt = (
    "You are preparing a minimal sample run for the Rulefile-Updater.\n"
    f"CHAT HISTORY JSONL NAME: {smallest_jsonl.name}\nJSONL CONTENT:\n{cleaned_jsonl_content}\n\n"
    "CLAUDE_MD_SNIPPETS (trimmed):\n" + "\n\n".join(claude_blurbs)
)
# print(prompt)

resp = generate_filepath_json(prompt=prompt, model='gpt-oss-120b')
resp


{'improvements': [{'filepath': 'modal_apps/CLAUDE.md',
   'improvement': '- After any change to a Modal app\'s `scaledown_window` (e.g., increased to keep the container warm longer), update this file with the new value and the rationale (e.g., "PDF bucket processor frequently invoked by v2_pipeline"), so future developers know the scaling intent.\n- When fixing import paths (e.g., changing `from v2_pipeline.utils.image_utils` to `from utils.image_utils`), add a note here that the import was migrated and that a repo‑wide search for stale `v2_pipeline` imports should be run to avoid similar ModuleNotFoundError.\n- Add a brief debugging tip for the `get_text_from_pdf_modal_app` endpoint: log the full request JSON (excluding any large base64 payload) before returning a 400, to surface missing or malformed fields.'},
  {'filepath': 'CLAUDE.md',
   'improvement': '- Add a CI/maintenance rule: run `grep -R "from v2_pipeline" .` after each merge and fail the build if any matches remain, preven